In [ ]:
from google.colab import drive
drive.mount('/content/drive')
% cd 'drive/My Drive/ece c247/project'
% ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/ece c247/project
bestM.pt                fake_y_train_valid.npy  X_test.npy
disc                    gen                     X_train_valid.npy
EEG_loading.ipynb       person_test.npy         y_test.npy
fake_X_train_valid.npy  person_train_valid.npy  y_train_valid.npy


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from torchvision import transforms, utils
import time
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")

X_train_valid /= 100
y_train_valid -= 769
print('y_train_valid', y_train_valid[:10])

# copy numpy data to tensor
X_train_valid_tensor = torch.from_numpy(X_train_valid).float().to(device)
y_train_valid_tensor = torch.from_numpy(y_train_valid).float().long().to(device) # do not forget .long()


cuda
y_train_valid [2 3 0 0 0 0 2 1 3 3]


In [ ]:
DROPOUT=0.5
class discriminator(nn.Module):
    def __init__(self,classes=4):
        super(discriminator,self).__init__()
        self.conv11 = nn.Conv2d(1, 25, (1, 5))
        self.conv12 = nn.Conv2d(25, 25, (22, 1))
        self.batchnorm1 = nn.BatchNorm2d(25)
        self.pool = nn.MaxPool2d((1,2))
        
        # Layer 2
        self.conv2 = nn.Conv2d(25, 50, (1, 5))
        self.batchnorm2 = nn.BatchNorm2d(50)
        
        # Layer 3

        self.conv3 = nn.Conv2d(50, 100, (1, 5))
        self.batchnorm3 = nn.BatchNorm2d(100)

        # Layer 4

        self.conv4 = nn.Conv2d(100, 200, (1, 5))
        self.batchnorm4 = nn.BatchNorm2d(200)

        
        # FC Layer
        self.fc_class = nn.Linear(200 * 58, 4)
        self.fc_source=nn.Linear(200 * 58, 1)

        self.sig=nn.Sigmoid()
        self.soft=nn.Softmax(dim=1)

    def forward(self,x):

        # Layer 1
        # reshape x: (B, 22, 1000) -> (B, 1, 22, 1000), B,C,H,W
        x = x.view(-1, 1, 22, 1000)

        x = self.conv12(self.conv11(x))
        x = self.batchnorm1(x)
        x = F.elu(x)
        x = self.pool(x)
        x = F.dropout(x, DROPOUT)
        
        # Layer 2
        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = F.elu(x)
        x = self.pool(x)
        x = F.dropout(x, DROPOUT)
        
        # Layer 3
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, DROPOUT)
        x = self.pool(x)
        
        # Layer 4
        x = self.conv4(x)
        x = self.batchnorm4(x)
        x = F.elu(x)
        x = self.pool(x)
        x = F.dropout(x, DROPOUT)

        x = x.reshape(-1, 200 * 58)

        rf=self.sig(self.fc_source(x))#checks source of the data---i.e.--data generated(fake) or from training set(real)
        c=self.soft(self.fc_class(x))#checks class(label) of data--i.e. to which label the data belongs in the dataset
        
        return rf,c 

In [ ]:
class generator(nn.Module):

    #generator model
    def __init__(self,in_channels):
        super(generator,self).__init__()
        self.fc1=nn.Linear(in_channels,384)
        # 1 -> 8
        self.t1=nn.Sequential(
            nn.ConvTranspose2d(in_channels=384,out_channels=192,kernel_size=(1,8),stride=1,padding=0),
            nn.BatchNorm2d(192),
            nn.ReLU()
        )
        # 8 -> 39
        self.t2=nn.Sequential(
            nn.ConvTranspose2d(in_channels=192,out_channels=96,kernel_size=(1,8),stride=(1,5),padding=(0,2)),
            nn.BatchNorm2d(96),
            nn.ReLU()
        )
        #41 -> 199
        self.t3=nn.Sequential(
            nn.ConvTranspose2d(in_channels=96,out_channels=48,kernel_size=(1,9),stride=(1,5),padding=0),
            nn.BatchNorm2d(48),
            nn.ReLU()
        )
        #199 -> 1000
        self.t4=nn.Sequential(
            nn.ConvTranspose2d(in_channels=48,out_channels=24,kernel_size=(22, 1),stride=1,padding=0),
            nn.BatchNorm2d(24),
            nn.ReLU()
        )
        #199 -> 1000
        self.t5=nn.Sequential(
            nn.ConvTranspose2d(in_channels=24,out_channels=1,kernel_size=(1,10),stride=(1,5),padding=0),
            nn.Tanh()
        )
    
    def forward(self,x):
      x=x.view(-1,110)
      x=self.fc1(x)
      x=x.view(-1,384,1,1)
      x=self.t1(x)
      # print(x.shape)
      x=self.t2(x)
      # print(x.shape)
      x=self.t3(x)
      # print(x.shape)
      x=self.t4(x)
      # print(x.shape)
      x=self.t5(x)
      # print(x.shape)
      return x #output of generator

In [ ]:
import torch
from torch import nn
import torch.utils.data
import torchvision.datasets as data_set
import torchvision.transforms as transforms
from torch import optim
from torch.autograd import Variable
import numpy as np
import torchvision.utils as vutils

lr=0.0002
epochs=100
batch_size=128


def compute_acc(preds, labels):
	correct = 0
	preds_ = preds.data.max(1)[1]
	correct = preds_.eq(labels.data).cpu().sum()
	acc = float(correct) / float(len(labels.data)) * 100.0
	return acc

def weights_init(m):
		classname = m.__class__.__name__
		if classname.find('Conv') != -1:
			m.weight.data.normal_(0.0, 0.02)
		elif classname.find('BatchNorm') != -1:
			m.weight.data.normal_(1.0, 0.02)
			m.bias.data.fill_(0)


class EEGDataset(Dataset):
    """EEG dataset."""
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
          pass 
            # x = self.transform(x)
            # y = self.transform(y)
        return x, y
        
    def __len__(self):
        return len(self.subset)

init_dataset = TensorDataset(X_train_valid_tensor, y_train_valid_tensor) 


train_data = EEGDataset(
    init_dataset, transform=None)


dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)



gen=generator(110).to(device)
disc=discriminator().to(device)

gen.apply(weights_init)

optimD=optim.Adam(disc.parameters(),lr)
optimG=optim.Adam(gen.parameters(),lr * 50)

source_obj=nn.BCELoss()#source-loss

class_obj=nn.NLLLoss()#class-loss


for epoch in range(epochs):
  for i,data in enumerate(dataloader,0):
    '''
    At first we will train the discriminator
    '''
    #training with real data----
    optimD.zero_grad()

    image,label=data
    image,label=image.to(device),label.to(device)

    batch_size = len(label)

    real_label = torch.FloatTensor(batch_size).to(device)
    real_label.fill_(1)

    fake_label = torch.FloatTensor(batch_size).to(device)
    fake_label.fill_(0)

    source_,class_=disc(image)#we feed the real images into the discriminator
    #print(source_.size())
    source_error=source_obj(source_.squeeze(),real_label)#label for real images--1; for fake images--0
    class_error=class_obj(class_,label)
    error_real=source_error+class_error
    error_real.backward()
    optimD.step()


    accuracy=compute_acc(class_,label)#getting the current classification accuracy

    #training with fake data now----


    noise_ = np.random.normal(0, 1, (batch_size, 110))#generating noise by random sampling from a normal distribution

    label=np.random.randint(0,4,batch_size)#generating labels for the entire batch

    noise=((torch.from_numpy(noise_)).float())
    noise=noise.to(device)#converting to tensors in order to work with pytorch

    label=((torch.from_numpy(label)).long())
    label=label.to(device)#converting to tensors in order to work with pytorch

    noise_image=gen(noise)
    #print(noise_image.size())

    source_,class_=disc(noise_image.detach())#we will be using this tensor later on
    #print(source_.size())
    source_error=source_obj(source_.squeeze(),fake_label)#label for real images--1; for fake images--0
    class_error=class_obj(class_,label)
    error_fake=source_error+class_error
    error_fake.backward()
    optimD.step()


    '''
    Now we train the generator as we have finished updating weights of the discriminator
    '''

    gen.zero_grad()
    source_,class_=disc(noise_image)
    source_error=source_obj(source_.squeeze(),real_label)#The generator tries to pass its images as real---so we pass the images as real to the cost function
    class_error=class_obj(class_,label)
    error_gen=source_error+class_error
    error_gen.backward()
    optimG.step()
    iteration_now = epoch * len(dataloader) + i


    print("Epoch--[{} / {}], Loss_Discriminator--[{}], Loss_Generator--[{}],Accuracy--[{}]".format(epoch,epochs,error_fake,error_gen,accuracy))


torch.save(gen, "./gen")
torch.save(disc, "./disc")



Epoch--[0 / 100], Loss_Discriminator--[1.3426823616027832], Loss_Generator--[-0.04933564364910126],Accuracy--[17.1875]
Epoch--[0 / 100], Loss_Discriminator--[2.398289203643799], Loss_Generator--[-0.04515790939331055],Accuracy--[29.6875]
Epoch--[0 / 100], Loss_Discriminator--[2.0384111404418945], Loss_Generator--[-0.005441993474960327],Accuracy--[26.5625]
Epoch--[0 / 100], Loss_Discriminator--[1.736771821975708], Loss_Generator--[0.094991534948349],Accuracy--[21.875]
Epoch--[0 / 100], Loss_Discriminator--[1.0726172924041748], Loss_Generator--[0.3245367109775543],Accuracy--[28.125]
Epoch--[0 / 100], Loss_Discriminator--[0.692634105682373], Loss_Generator--[0.44485390186309814],Accuracy--[30.46875]
Epoch--[0 / 100], Loss_Discriminator--[0.6171435713768005], Loss_Generator--[0.5003464221954346],Accuracy--[31.25]
Epoch--[0 / 100], Loss_Discriminator--[0.685440182685852], Loss_Generator--[0.39153847098350525],Accuracy--[24.21875]
Epoch--[0 / 100], Loss_Discriminator--[0.9685072302818298], Lo

In [ ]:
fake_sample_size = 2116
noise_ = np.random.normal(0, 1, (fake_sample_size, 110))#generating noise by random sampling from a normal distribution

label=np.random.randint(0,4,fake_sample_size)#generating labels for the entire batch

noise=((torch.from_numpy(noise_)).float())
noise=noise.to(device)#converting to tensors in order to work with pytorch

label=((torch.from_numpy(label)).long())
label=label.to(device)#converting to tensors in order to work with pytorch

model = torch.load("./gen").to(device)
model.eval()

noise_image=model(noise)

print(noise_image.shape)
print(label.shape)

label += 769
noise_image *= 100

fake_X_train_valid = noise_image.squeeze().detach().cpu().numpy()
fake_y_train_valid = label.detach().cpu().numpy()
np.save("fake_X_train_valid.npy", fake_X_train_valid)
np.save("fake_y_train_valid.npy", fake_y_train_valid)

torch.Size([2116, 1, 22, 1000])
torch.Size([2116])
